<a href="https://colab.research.google.com/github/hrushikute/DataAnalytics/blob/master/LSTM_IMDBSubwords8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LSTM on IMDB dataset**


## Downaload the data

In [4]:
import tensorflow_datasets as tfds
# Download the subword encoded pretokenized data set
dataset, info = tfds.load("imdb_reviews/subwords8k", with_info=True, as_supervised=True)


In [7]:
dataset ,info

({'test': <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
  'train': <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
  'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>},
 tfds.core.DatasetInfo(
     name='imdb_reviews',
     version=1.0.0,
     description='Large Movie Review Dataset.
 This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
     homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
     features=FeaturesDict({
         'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
   

In [8]:
# Get the tokenizer
tokenizer = info.features['text'].encoder

In [11]:
BUFFETR_SIZE = 10000
BATCH_SIZE = 256

# Get traina nd test split
train_data, test_data = dataset['train'],dataset['test']
print(len(train_data), len(test_data))
print(type(train_data))

25000 25000
<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [12]:
# Shuffle the training data
train_dataset = train_data.shuffle(BUFFETR_SIZE)

# Batch and Padd the data

train_dataset = train_dataset.padded_batch(BATCH_SIZE)
test_dataset = test_data.padded_batch(BATCH_SIZE)

### Build and Compile the model

In [18]:
import tensorflow as tf
import numpy as np

# Hyper parameters
batch_size=1
timesteps = 20
features = 16
lstm_dim = 8

# lets have array with random values
random_input = np.random.rand(batch_size, timesteps, features)
print(f'Shape of array {random_input.shape}')


# Define LSTM that return single output.
lstm=tf.keras.layers.LSTM(lstm_dim)
result = lstm(random_input)
print(f'Result : {result}')

# Define LSTM that retruns a sequence


Shape of array (1, 20, 16)
Result : [[-0.11051227  0.31303272  0.25312325 -0.437785    0.12659463 -0.17075601
  -0.12276196  0.00838692]]
